In [2]:
import pandas as pd

import sys

# setting path
sys.path.append('..')

from evaluation import get_time_step_fleet
from utils import load_solution

In [11]:
solution = load_solution("../data/solution_example.json")
df = get_time_step_fleet(solution=solution, ts=1)

In [14]:
df.columns

Index(['datacenter_id', 'server_generation', 'server_id', 'action'], dtype='object')